# 1. 문자열을 사용한 작업
## 5) 유사 척도

NLP에서 태거, 청커 등의 성능을 테스트하기 위해, 정보 검색에서 검색된 표준 점수를 사용할 수 있음<br>
훈련 파일에서 얻은 표준 점수를 사용한 개체명 인식기의 출력을 분석하는 방법

In [1]:
from __future__ import print_function

In [2]:
from nltk.metrics import *

In [3]:
training = 'PERSON OTHER PERSON OTHER OTHER ORGANIZATION'.split()

In [8]:
testing = 'PERSON OTHER OTHER OTHER OTHER OTHER'.split()

In [9]:
print(accuracy(training, testing))

0.6666666666666666


In [10]:
trainset = set(training)

In [11]:
testset = set(testing)

In [12]:
precision(trainset, testset)

1.0

In [13]:
print(recall(trainset, testset))

0.6666666666666666


In [14]:
print(f_measure(trainset, testset))

0.8


편집 거리 알고리즘을 사용한 유사 척도

* 첫 번째 문자열에서 두 번째 문자열로 문자 복사(비용 0) 및 다른 것으로 문자 대체(비용 1) : D(i-1, j-1) + d(si, tj) (대체/복사)
* 첫 번째 문자열의 문자 삭제(비용 1) : D(i, j-1) + 1(삭제)
* 두 번째 문자열의 문자 삽입(비용 1): D(i, j) = min D(i-1, j) + 1(삽입)

In [16]:
from __future__ import print_function
def _edit_dist_init(len1, len2):
    lev = []
    for i in range(len1):
        lev.append([0] * len2) # 2차원 배열을 0으로 초기화
    for i in range(len1):
        lev[i][0] = i # 열 0: 0, 1, 2, 3, 4
    for j in range(len2):
        lev[0][j] = j # 행 0: 0, 1, 2, 3, 4
    return lev

def _edit_dist_step(lev, i, j, s1, s2, transpositions=False):
    c1 = s1[i-1]
    c2 = s2[j-1]
    
    # s1에서 문자 스킵
    a = lev[i-1][j] + 1
    # s2에서 문자 스킵
    b = lev[i][j-1] + 1
    # 대체
    c = lev[i-1][j-1] + (c1!=c2)
    # 이항
    d = c + 1
    if transpositions and i > 1 and j > 1:
        if s1[i-2] == c2 and s2[j-2] == c1:
            d = lev[i-2][j-2] + 1
    # 최소값 선택
    lev[i][j] = min(a, b, c, d)
    
def edit_distance(s1, s2, transpositions=False):
    # 2차원 배열 설정
    len1 = len(s1)
    len2 = len(s2)
    lev = _edit_dist_init(len1 + 1, len2 + 1)
    
    # 배열을 반복
    for i in range(len1):
        for j in range(len2):
            _edit_dist_step(lev, i+1, j+1, s1, s2, transpositions=transpositions)
    
    return lev[len1][len2]

In [17]:
import nltk

In [18]:
from nltk.metrics import *

In [19]:
edit_distance("relate", "relation")

3

In [20]:
edit_distance("suggestion", "calculation")

7

자카드 계수를 사용한 유사 척도

In [21]:
def jacc_similarity(query, document):
    first = set(query).intersection(set(document))
    second = set(query).union(set(document))
    
    return len(first)/len(second)

In [22]:
import nltk

In [23]:
from nltk.metrics import *

In [24]:
X = set([10, 20, 30, 40])

In [25]:
Y = set([20, 30, 60])

In [26]:
print(jaccard_distance(X, Y))

0.6


스미스 워터맨 거리를 사용한 유사 척도 적용

스밋 워터맨 거리는 할당되는 비용과 비용 값(대체)으로 매핑하는 알파벳 함수로 구성됨.<br>
비용은 또한 G에 할당된다.

1. 0 // 다시 시작
2. D(i-1, j-1) - d(si, tj) // 대체/복사
3. D(i, j) = max D(i-1, j) - G // 삽입
4. D(i, j-1) - G // 삭제
5. G = 1 // 갭에 대한 예제 값
6. d(c, c) = -2 // 문맥 의존 대체 비용
7. d(c, d) = +1 // 문맥 의존 대체 비용

이진 거리 메트릭

In [27]:
def binary_distance(label1, label2):
    return 0.0 if label1 == label2 else 1.0

In [28]:
import nltk

In [29]:
from nltk.metrics import *

In [30]:
X = set([10, 20, 30, 40])

In [31]:
Y = set([30, 50, 70])

In [32]:
binary_distance(X, Y)

1.0